<a href="https://colab.research.google.com/github/AdrianduPlessis/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1) Load and clean the data (or determine the best method to drop observations when running tests)

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

##1) Load and clean the data (or determine the best method to drop observations when running tests)

In [2]:
headings = ['Democrat/Republican',
            'handicapped-infants',
            'water-project-cost-sharing',
            'adoption-of-the-budget-resolution',
            'physician-fee-freeze',
            'el-salvador-aid',
            'religious-groups-in-schools',
            'anti-satellite-test-ban',
            'aid-to-nicaraguan-contras',
            'mx-missile',
            'immigration',
            'synfuels-corporation-cutback',
            'education-spending',
            'superfund-right-to-sue',
            'crime',
            'duty-free-exports',
            'export-administration-act-south-africa']

voting_issues = ['handicapped-infants',
            'water-project-cost-sharing',
            'adoption-of-the-budget-resolution',
            'physician-fee-freeze',
            'el-salvador-aid',
            'religious-groups-in-schools',
            'anti-satellite-test-ban',
            'aid-to-nicaraguan-contras',
            'mx-missile',
            'immigration',
            'synfuels-corporation-cutback',
            'education-spending',
            'superfund-right-to-sue',
            'crime',
            'duty-free-exports',
            'export-administration-act-south-africa']
dirty_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None, names=headings, na_values='?')
dirty_df.head()

,Democrat/Republican,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [52]:
#convert to bool values
conversion_dict = {'n' : False, 'y' : True}

bool_df = pd.DataFrame(columns=[headings])
bool_df['Democrat/Republican'] = dirty_df['Democrat/Republican']

for issue in voting_issues:
  bool_df[issue] = dirty_df[issue].map(conversion_dict)
 
##########Data types not changing arggggg!
#change dem/rep col from obj to str
#bool_df = bool_df.astype({'Democrat/Republican' : 'sting'})

bool_df = bool_df.astype({'Democrat/Republican' : 'string', 'handicapped-infants' : 'bool'})

bool_df.head()

,Democrat/Republican,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,False,True,False,True,True,True,False,False,False,True,NaN,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,NaN
2,democrat,NaN,True,True,NaN,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,NaN,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,NaN,True,True,True,True


In [51]:
bool_df.dtypes

Democrat/Republican                       object
handicapped-infants                       object
water-project-cost-sharing                object
adoption-of-the-budget-resolution         object
physician-fee-freeze                      object
el-salvador-aid                           object
religious-groups-in-schools               object
anti-satellite-test-ban                   object
aid-to-nicaraguan-contras                 object
mx-missile                                object
immigration                               object
synfuels-corporation-cutback              object
education-spending                        object
superfund-right-to-sue                    object
crime                                     object
duty-free-exports                         object
export-administration-act-south-africa    object
dtype: object

In [46]:
#seperate into two samples: Rep/Dem
republican_subset = bool_df[bool_df['Democrat/Republican']=='republican']
democrat_subset = bool_df[bool_df['Democrat/Republican']=='democrat']

#sanity check
assert (len(republican_subset)+len(democrat_subset)==len(bool_df))

ValueError: ignored

In [0]:
republican_subset['handicapped-infants'].fillna(republican_subset['handicapped-infants'].mode())

0        n
1        n
7        n
8        n
10       n
11       n
14       n
15       n
18       n
28       y
30       n
33       n
35       n
36       y
37       y
38       n
49       n
51       n
53       y
55       n
56       n
57       n
58       n
59       n
61       n
65       y
66       n
67       n
71       y
73       y
      ... 
356      n
357      n
359      n
363      y
364      y
369      n
374      n
377      n
378      n
379      n
392      y
393    NaN
399      n
400      n
401      n
402    NaN
403      n
404      y
405      n
409      n
410      n
412      n
413      y
416      y
420      y
427      n
430      n
432      n
433      n
434      n
Name: handicapped-infants, Length: 168, dtype: object

In [0]:
def fill_na_of_cols_with_mode(df, cols):
  '''
  Takes a dataframe and list of columns
  Returns dataframe with each specified column's na's filled with the mode of that column
  '''
  filled_df = pd.DataFrame(columns=cols)
  for col in cols:
    filled_df[col] = df[col].fillna(df[col].mode())
  return filled_df

voting_issues = ['handicapped-infants',
            'water-project-cost-sharing',
            'adoption-of-the-budget-resolution',
            'physician-fee-freeze',
            'el-salvador-aid',
            'religious-groups-in-schools',
            'anti-satellite-test-ban',
            'aid-to-nicaraguan-contras',
            'mx-missile',
            'immigration',
            'synfuels-corporation-cutback',
            'education-spending',
            'superfund-right-to-sue',
            'crime',
            'duty-free-exports',
            'export-administration-act-south-africa']

In [0]:
fill_na_of_cols_with_mode(republican_subset, voting_issues)

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
7,n,y,n,y,y,y,n,n,n,n,n,n,y,y,NaN,y
8,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
10,n,y,n,y,y,n,n,n,n,n,NaN,NaN,y,y,n,n
11,n,y,n,y,y,y,n,n,n,n,y,NaN,y,y,NaN,NaN
14,n,y,n,y,y,y,n,n,n,n,n,y,NaN,NaN,n,NaN
15,n,y,n,y,y,y,n,n,n,y,n,y,y,NaN,n,NaN
18,n,y,n,y,y,y,n,n,n,n,n,NaN,y,y,n,n
28,y,n,n,y,y,n,y,y,y,n,n,y,y,y,n,y


##2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01